# Comparison of spectrum between eNATL60 outputs and Altika Satellite for Region 1 and ASO months

Requisites :
 - notebook process-spectrum-comparison-eNATL60-region1-Altkia-ASO has successfully run and produced all the necessary result_*.nc in results_Altika-eNATL60-Region1-ASO
 - gonzag_cloud
 - climporn
 


In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud/gonzag' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(GONZAG_DIR)
import gonzag as gz


In [3]:
import xarray as xr
import sys
import glob
import numpy as nmp
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

%matplotlib inline


In [4]:
CLIMPORN_DIR = '/home/jovyan/climporn/python' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(CLIMPORN_DIR)
import climporn as cp


### Params

For the model

In [5]:
model = 'GIGATL'
region = '1'
season = 'fma'
name_mod = model+'-Region'+region+'-'+season
name_ssh_mod = 'zeta_bl'


For altimetry data

In [6]:
name_sat= 'Altika'
name_ssh_sat='sla_unfiltered'


### Data

In [7]:
fresults=sorted(glob.glob('results_'+name_sat+'-'+name_mod+'/result_??.nc'))
dsn=xr.open_mfdataset(fresults,concat_dim='time',combine='nested')

In [8]:
#clean up some remaining Nans

ds=dsn.where(nmp.isnan(dsn[name_ssh_sat])==0, drop=True)

In [9]:
print(ds[name_ssh_mod].values)

[-9999. -9999. -9999. ... -9999. -9999. -9999.]


### Maps of the tracks

In [10]:
fig = plt.figure(num = 1, figsize=(15,9), facecolor='w', edgecolor='k')
ax = plt.subplot(111,projection=ccrs.PlateCarree())
ax.coastlines(resolution="10m")
ax.set_xticks(nmp.arange(-80,30,1.5), crs=ccrs.PlateCarree())
ax.set_xticklabels(nmp.arange(-80,30,1.5))
ax.set_yticks(nmp.arange(-50,50,1.5), crs=ccrs.PlateCarree())
ax.set_yticklabels(nmp.arange(-50,50,1.5))

lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
gl = ax.gridlines(draw_labels=True, linestyle=':', color='black',
                          alpha=0.5)

idx = ds.longitude
idy = ds.latitude
t = ds.time_counter
cf = ax.scatter(idx, idy, c=t, cmap = 'viridis', marker='.', s=3 )
    
cbar = plt.colorbar(cf,shrink=0.8)
cbar.set_ticks([])

plt.savefig('plots/tracks_'+name_sat+'-'+name_mod+'.png')
del fig,ax

### Time series of SSH 

In [11]:
clr_sat = '#AD0000'
clr_mod = '#008ab8'

VT = ds.time_counter ; 

fig = plt.figure(num = 1, figsize=(25,15), facecolor='w', edgecolor='k')
ax = plt.subplot(111)
plt.hlines(0,nmp.min(ds.time_counter.values),nmp.max(ds.time_counter.values), colors='k',label=None,  zorder=5)
plt.plot(VT, ds[name_ssh_sat]-nmp.mean(ds[name_ssh_sat]), '.', color=clr_sat, markersize=5, 
         alpha=0.5, label=name_sat, zorder=10)
plt.plot(VT, ds[name_ssh_mod]-nmp.mean(ds[name_ssh_mod]), '.', color=clr_mod, markersize=5, 
         alpha=0.5, label=name_mod, zorder=15)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.xticks(rotation='60')
plt.ylabel('SSH [m]')
ax.grid(color='k', linestyle='-', linewidth=0.3)
lgnd = plt.legend(bbox_to_anchor=(0.55, 1.), ncol=1, shadow=True, fancybox=True)

plt.savefig('plots/SSH_'+name_sat+'-'+name_mod+'.png')

### Spectrum

In [12]:
%%time

ISeg_beg, ISeg_end = gz.FindUnbrokenSegments( ds.time, ds.distance, ds[name_ssh_mod],\
                                             rcut_time=1.2e+09, rcut_dist=7.8 )
NbSeg, Nsl, IDEDSeg = gz.SegmentSelection(ISeg_beg, ISeg_end, np_valid_seg=70)
XPs, XPm, rdist_sample = gz.Process4FFT( IDEDSeg, ds.distance, ds[name_ssh_mod], ds[name_ssh_sat] )
Kwn, PwSpc_s, PwSpc_m = gz.ApplyFFT( IDEDSeg, XPs, XPm, rdist_sample )

# Building our spectrum as the mean of the NbSeg spectra:
vps_mod = nmp.mean(PwSpc_m[:,:],axis=0)
vps_sat = nmp.mean(PwSpc_s[:,:],axis=0)



KeyboardInterrupt: 

In [13]:
# Blabla for the plot:
cinfrm = str(NbSeg)+' segments\n'+str(Nsl)+' points/segment\n'+r'$\Delta$d sat.: '+str(round(rdist_sample,1))+' km'

ii = cp.plot("pow_spectrum_ssh")(Kwn, vps_mod, clab1=name_mod+' ("'+name_ssh_mod+'")', clr1=clr_mod, lw1=5, \
                                 cinfo=cinfrm, logo_on=False, \
                                 L_min=13., L_max=500., P_min_y=-6, P_max_y=1, \
                                 l_show_k4=False, l_show_k5=True, l_show_k11o3=False, l_show_k2=True, \
                                 vk2=Kwn, vps2=vps_sat, clab2=name_sat+' ("'+name_ssh_sat+'")', clr2=clr_sat, lw2=4)

plt.savefig('plots/spectrum_SSH_'+name_sat+'-'+name_mod+'.png')


NameError: name 'NbSeg' is not defined

In [ ]:
del fig,ax